<a href="https://colab.research.google.com/github/r-y-a-n-m-a-r-t-i-n/CIS5190_Project/blob/main/Project_Amazon_Fine_Food_Reviews_jbr_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Some imports that you might need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset, DataLoader, TensorDataset
import nltk
import os

# Switch PyTorch to GPU compute
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("We're using:", device)

We're using: cuda


Hi Jack

For the data, you can click download at
https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews

You can choose to store the data in your google drive and mount your drive for accessing them from the following cell.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Or you can download it by the following cell:

In [ ]:
!pip install --upgrade --no-cache-dir gdown
if not os.path.exists("Reviews.csv"):
    !gdown 1_kLSwiRYtiXF7h9V1FlTqTapOHiYU5Mk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6
Downloading...
From (uriginal): https://drive.google.com/uc?id=1_kLSwiRYtiXF7h9V1FlTqTapOHiYU5Mk
From (redirected): https://drive.google.com/uc?id=1_kLSwiRYtiXF7h9V1FlTqTapOHiYU5Mk&confirm=t&uuid=b2c1521a-1662-4559-8b40-7bf102022947
To: /content/Reviews.csv
100% 301M/301M [00:01<00:00, 171MB/s]


In [ ]:
# Remeber to change the path here for the corresponding files you need
df = pd.read_csv('Reviews.csv')
print(df.shape)
df = df.head(10000)
print(df.shape)

(568454, 10)
(10000, 10)


In [ ]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
# import libraries for preprocessing
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.sentiment import SentimentAnalyzer
from sklearn.preprocessing import MultiLabelBinarizer
from string import punctuation
from string import digits
from collections import Counter
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# function to remove stop words
eng_stopwords = stopwords.words('english') 
def filter_words(alist):
  ret_l = []
  for w in alist:
    for el in w:
        if el in punctuation:
          w = w.replace(el,"")
    if w.lower() not in eng_stopwords and w not in punctuation and w != '...' and w !='br' and w not in digits and w != 'nt':
      ret_l.append(w.lower())
  return ret_l

In [ ]:
# preprocess data

df['tokenized_text'] = df['Text'].apply(word_tokenize)
df['filtered_text'] = df['tokenized_text'].apply(filter_words)
all_reviews = df['filtered_text'].tolist()
all_words=[element for sublist in all_reviews for element in sublist]
total_words=len(all_words)
count_words = Counter(all_words)
sorted_words=count_words.most_common(total_words)
sorted_words = sorted_words[:15000]
print(sorted_words[:10])

[('like', 4290), ('coffee', 3678), ('good', 3477), ('taste', 3001), ('one', 2903), ('great', 2860), ('flavor', 2681), ('product', 2516), ('would', 2308), ('love', 2043)]


In [ ]:
# one hot encode the words
vocab_to_int={w:i+1 for i,(w,c) in enumerate(sorted_words)}
print(vocab_to_int)


{'like': 1, 'coffee': 2, 'good': 3, 'taste': 4, 'one': 5, 'great': 6, 'flavor': 7, 'product': 8, 'would': 9, 'love': 10, 'really': 11, 'food': 12, 'tea': 13, 'much': 14, 'get': 15, 'use': 16, 'also': 17, 'best': 18, 'little': 19, 'tried': 20, 'time': 21, 'buy': 22, 'price': 23, 'make': 24, 'amazon': 25, 'better': 26, 'find': 27, 'try': 28, 'well': 29, 'cup': 30, 'even': 31, 'chips': 32, 'drink': 33, 'water': 34, 'first': 35, 'sugar': 36, 'could': 37, 'chocolate': 38, 'mix': 39, 'found': 40, 'eat': 41, 'made': 42, 'bag': 43, 'hot': 44, 'used': 45, 'bought': 46, 'sweet': 47, 'flavors': 48, 'favorite': 49, 'recommend': 50, 'way': 51, 'still': 52, 'many': 53, 'think': 54, 'box': 55, 'since': 56, 'order': 57, 'juice': 58, 'milk': 59, 'tastes': 60, 'nice': 61, 'dog': 62, 'bit': 63, 'two': 64, 'want': 65, 'delicious': 66, 'store': 67, 'go': 68, 'know': 69, 'got': 70, 'brand': 71, 'give': 72, 'free': 73, 'say': 74, 'without': 75, 'something': 76, 'pack': 77, 'never': 78, 'different': 79, 'easy

In [ ]:
# transform reviews into lists of numbers
e_rev = []
for review in all_reviews:
  e = []
  for word in review:
    if word not in vocab_to_int.keys():
      e.append(0)
    else:
      e.append(vocab_to_int[word])
  e_rev.append(e)

In [ ]:
# make the feature array and make the inputs all the same length
length = 100
feats=np.zeros((len(e_rev), length), dtype=int)

for i, rev in enumerate(e_rev):
  if len(rev) > length:
    feats[i,:] = np.array(rev[:length])
  else:
    for j in range(len(rev)):
      feats[i,j] = rev[j]
# X_train
feats

array([[   46,   202,  6858, ...,     0,     0,     0],
       [    8,   255,  2164, ...,     0,     0,     0],
       [ 4809,   211, 11206, ...,     0,     0,     0],
       ...,
       [  285,  4997, 20957, ...,     0,     0,     0],
       [   10,   294,    15, ...,     0,     0,     0],
       [  400,   116,   992, ...,     0,     0,     0]])

In [ ]:
# creature training
df['Score']
labels = df['Score'].array
# y_train
# labels = labels / 5

# labels = labels.to_numpy()

print(labels.shape)
print(feats.shape)
# feats = feats.T
data = np.column_stack((feats, labels))

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(0, 1)])

labels = transform(data)

(10000,)
(10000, 100)


In [ ]:
# Split into Train, Validation, Test

feats = data[:, :-1]
labels = data[:, -1]

train_x=feats[:int(0.8*len(feats))]
train_y=labels[:int(0.8*len(feats))]
valid_x=feats[int(0.8*len(feats)):int(0.9*len(feats))]
valid_y=labels[int(0.8*len(feats)):int(0.9*len(feats))]
test_x=feats[int(0.9*len(feats)):]
test_y=labels[int(0.9*len(feats)):]
print(train_x.shape, valid_y.shape, test_y.shape)

(8000, 100) (1000,) (1000,)


In [ ]:
# Split the Dataset

train_dataset = TensorDataset(torch.FloatTensor(train_x), torch.FloatTensor(train_y))
valid_dataset = TensorDataset(torch.FloatTensor(valid_x), torch.FloatTensor(valid_y))
test_dataset = TensorDataset(torch.FloatTensor(test_x), torch.FloatTensor(test_y))

batch_size = 5

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Convolutional Neural Network Class
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # Create the layers of CNN here -- Currently from HW5 so may want to change to tune

        # Convolutional Layers & Pooling
        self.conv1 = nn.Conv1d(batch_size, 12, 5)
        self.pool = nn.MaxPool1d(3)
        self.conv2 = nn.Conv1d(12, batch_size, 5)
        # Fully Connected Layers
        self.fc1 = nn.Linear(9, 32)
        self.fc2 = nn.Linear(32, 26)
        self.fc3 = nn.Linear(26, 14)
        self.fc4 = nn.Linear(14, 1)
        self.sigmoid=nn.Sigmoid()

        # STUDENT TODO END

    def forward(self, x):
        # Perform the forward pass through the layers -- also still from HW5

        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        x = self.sigmoid(x)
        x = x.view(batch_size, -1)
        x = x[:, -1]
        return x

# Create an instance of Net and move it to the GPU
model = Net()
model.to(device)

Net(
  (conv1): Conv1d(5, 12, kernel_size=(5,), stride=(1,))
  (pool): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(12, 5, kernel_size=(5,), stride=(1,))
  (fc1): Linear(in_features=9, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=26, bias=True)
  (fc3): Linear(in_features=26, out_features=14, bias=True)
  (fc4): Linear(in_features=14, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [ ]:
# 1. Set the criterion to be cross entropy loss
criterion = nn.BCELoss()

# 2. Experiment with different optimizers
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# torch.backends.cudnn.enabled = False

In [ ]:
train_loss, validation_loss = [], []
train_acc, validation_acc = [], []

# Note that we have set the number of epochs to be 10. You can choose to increase or decrease the number of epochs.
num_epochs = 5
for epoch in range(num_epochs):
    
    model.train()
    running_loss = 0.
    correct, total = 0, 0 
    
    for i, data in enumerate(train_dataloader, 0):

        inputs, labels = data
        # print(labels)
        # labels = labels.type(torch.LongTensor)
        # 1. Store the inputs and labels in the GPU
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # 2. Get the model predictions
        predictions = model(inputs).reshape((batch_size,))

        # 3. Zero the gradients out
        optimizer.zero_grad()

        # 4. Get the loss
        # print(predictions)
        # input(labels)
        loss = criterion(predictions, labels)

        # 5. Calculate the gradients
        loss.backward()

        # 6. Update the weights
        optimizer.step()
            
        running_loss += loss.item()

        predicted = predictions

        # print("predicted: ", predicted)
        # print("labels: ", labels)

        # _, predicted = torch.max(predictions, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    train_loss.append(running_loss / len(train_dataloader))
    train_acc.append(correct/total)
            
    model.eval()
    running_loss = 0.
    correct, total = 0, 0 
    
    for i, data in enumerate(val_dataloader, 0):

        inputs, labels = data
        # labels = labels.type(torch.LongTensor)
        # 1. Store the inputs and labels in the GPU
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # 2. Get the model predictions
        predictions = model(inputs)

        # 3. Get the loss
        loss = criterion(predictions, labels)

        running_loss += loss.item()

        predicted = predictions

        
        # _, predicted = torch.max(predictions, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    validation_loss.append(running_loss / len(val_dataloader))
    validation_acc.append(correct/total)

    print(f"Epoch {epoch+1}:")

    print(f"Training Loss:", round(train_loss[epoch], 3))
    print(f"Validation Loss:", round(validation_loss[epoch], 3))
    
    print(f"Training Accuracy:", round(train_acc[epoch], 3))
    print(f"Validation Accuracy:", round(validation_acc[epoch], 3))

    print("------------------------------")

Epoch 1:
Training Loss: -311.734
Validation Loss: -310.7
Training Accuracy: 0.09
Validation Accuracy: 0.095
------------------------------
Epoch 2:
Training Loss: -314.925
Validation Loss: -310.7
Training Accuracy: 0.092
Validation Accuracy: 0.095
------------------------------
Epoch 3:
Training Loss: -314.881
Validation Loss: -310.7
Training Accuracy: 0.092
Validation Accuracy: 0.095
------------------------------
Epoch 4:
Training Loss: -314.925
Validation Loss: -310.7
Training Accuracy: 0.092
Validation Accuracy: 0.095
------------------------------
Epoch 5:
Training Loss: -314.925
Validation Loss: -310.7
Training Accuracy: 0.092
Validation Accuracy: 0.095
------------------------------


In [ ]:
model.eval()

test_predictions = np.array([])

for i, data in enumerate(test_dataloader, 0):

    inputs = data
    print(inputs)

    # STUDENT TODO START:
    # 1. Store the inputs in the GPU
    inputs = inputs.to(device)
    
    # 2. Get the model predictions
    predictions = model(inputs)
    # STUDENT TODO END

    _, predicted = torch.max(predictions, 1)

    test_predictions = np.concatenate((test_predictions, predicted.detach().cpu().numpy()))

[tensor([[1.7300e+02, 9.6800e+02, 1.1100e+02, 1.0000e+00, 2.5000e+02, 2.0000e+00,
         4.8000e+01, 4.7000e+01, 4.1400e+02, 5.5300e+02, 5.0000e+00, 6.8020e+03,
         4.0000e+00, 5.1750e+03, 1.8800e+02, 1.8740e+03, 7.0000e+00, 2.5000e+01,
         6.0000e+00, 2.3000e+01, 7.6000e+01, 1.6500e+02, 5.0000e+01, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000

AttributeError: ignored